# Chapter 6: Set Operators — AdventureWorks2019

This notebook contains ten propositions using **UNION / UNION ALL / INTERSECT / EXCEPT** across the `Production`, `Person`, `Purchasing`, and `Sales` schemas in **AdventureWorks2019**. Each proposition includes a short business value explanation and a runnable SQL snippet.

> **Note:** In DBeaver or Azure Data Studio, set the active database to `AdventureWorks2019` before running the queries.

## Proposition 1: Identify products that exist in inventory but have never appeared in a transaction record
**Business Value:** Helps detect inactive or obsolete inventory items that aren’t selling and may need to be discounted, written off, or discontinued.

In [ ]:


SELECT pi.ProductID
FROM Production.ProductInventory AS pi
EXCEPT
SELECT th.ProductID
FROM Production.TransactionHistory AS th
ORDER BY ProductID;

## Proposition 2: Show all products that either have a model assigned or none at all
**Business Value:** Helps the company see which products have detailed info (a model) and which still need it added.

In [ ]:

SELECT p.ProductID, p.Name, 'HasModel' AS ModelStatus
FROM Production.Product AS p
WHERE p.ProductModelID IS NOT NULL
UNION
SELECT p.ProductID, p.Name, 'NoModel' AS ModelStatus
FROM Production.Product AS p
WHERE p.ProductModelID IS NULL
ORDER BY ModelStatus, Name;

## Proposition 3: Group products by whether they have a selling price or not
**Business Value:** Quickly highlights pricing completeness so the team can see which items are ready to sell and which still need a price.

In [ ]:

SELECT ProductID, Name, ListPrice, 'Priced' AS PriceStatus
FROM Production.Product
WHERE ListPrice > 0
UNION ALL
SELECT ProductID, Name, ListPrice, 'NoListPrice' AS PriceStatus
FROM Production.Product
WHERE (ListPrice = 0 OR ListPrice IS NULL)
ORDER BY PriceStatus, Name;

## Proposition 4: People with an email address but no phone number
**Business Value:** Contact enrichment — helps prioritize who needs phone outreach.

In [ ]:

SELECT e.BusinessEntityID
FROM Person.EmailAddress AS e
EXCEPT 
SELECT p.BusinessEntityID
FROM Person.PersonPhone AS p
ORDER BY BusinessEntityID;

## Proposition 5: Products that have both a photo and customer reviews
**Business Value:** Great candidates for featured pages because they have rich media and social proof.

In [ ]:

SELECT DISTINCT ppp.ProductID
FROM Production.ProductProductPhoto AS ppp
INTERSECT
SELECT DISTINCT pr.ProductID
FROM Production.ProductReview AS pr
ORDER BY ProductID;

## Proposition 6: Products in the catalog without a photo
**Business Value:** Identifies products missing visual assets so listings can be improved for better conversions.

In [ ]:

SELECT p.ProductID, p.Name
FROM Production.Product AS p
EXCEPT
SELECT ppp.ProductID, p.Name
FROM Production.ProductProductPhoto AS ppp
JOIN Production.Product AS p ON p.ProductID = ppp.ProductID
ORDER BY p.ProductID;

## Proposition 7: Products listed but never ordered (via purchase orders)
**Business Value:** Reveals catalog items that have never been ordered — candidates for promotion, repricing, or retirement.

In [ ]:

SELECT p.ProductID, p.Name
FROM Production.Product AS p
EXCEPT
SELECT pod.ProductID, p.Name
FROM Purchasing.PurchaseOrderDetail AS pod
JOIN Production.Product AS p ON p.ProductID = pod.ProductID
ORDER BY p.ProductID;

## Proposition 8: Products present in purchasing and transaction history
**Business Value:** Shows items that are actively moving through the business (bought and transacted).

In [ ]:

SELECT pod.ProductID
FROM Purchasing.PurchaseOrderDetail AS pod
INTERSECT
SELECT th.ProductID
FROM Production.TransactionHistory AS th
ORDER BY ProductID;

## Proposition 9: Purchased products without a list price
**Business Value:** Finds items the company buys but hasn’t set a selling price for yet, so pricing can be fixed before selling.

In [ ]:

SELECT pod.ProductID
FROM Purchasing.PurchaseOrderDetail AS pod
EXCEPT
SELECT p.ProductID
FROM Production.Product AS p
WHERE p.ListPrice > 0
ORDER BY ProductID;

## Proposition 10: Customers that exist but have never placed an order
**Business Value:** Helps marketing target registered customers who haven’t converted yet.

In [ ]:

SELECT c.CustomerID
FROM Sales.Customer AS c
EXCEPT
SELECT soh.CustomerID
FROM Sales.SalesOrderHeader AS soh
ORDER BY CustomerID;

_Note: Some queries (2, 3, 7, 9) were refined with the help of an LLM (ChatGPT). Final testing and execution were done by me in DBeaver._